## Imports

In [1]:
# Data Privacy Final Project
# Jordan Bourdeau, Casey Forey
import numpy as np
import os
import pandas as pd
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import zipfile

## Data

In [70]:
url: str = 'https://jbourde2.w3.uvm.edu/data-privacy/data.zip'
file_path: str = 'data/powerlifting-data.csv'

# If the .zip file doesn't already exist, download it from the Silk server.
if not os.path.exists('data.zip'):
    try:
        r = requests.get(url, allow_redirects=True)
        print('Downloading zip file from server')
        open('data.zip', 'wb').write(r.content)
        print('Zip file downloaded from server')
    except Exception as e:
        print(e)
        print('Unable to download zip file from remote server')
        exit(1)

# If the data folder doesn't already exist, unzip the data zip
if not os.path.exists('data/'):
    try:
        with zipfile.ZipFile('data.zip') as zip:
            zip.extractall()
        print('Zip file extracted')
        df = pd.read_csv(file_path)
        print('Data read in')
    except Exception as e:
        print(e)
        print('No zip file to extract from')
        exit(1)

print('Loading dataframe')
df: pd.DataFrame = pd.read_csv(file_path)
# Drop unneeded columns
df = df.drop(['BirthYearClass', 'Division', 'AgeClass', 'Dots', 'Wilks', 'Glossbrenner', 'Goodlift', 
                'Federation', 'MeetCountry', 'MeetState', 'MeetTown', 'WeightClassKg',
                'Squat4Kg', 'Bench4Kg', 'Deadlift4Kg',], axis=1)
print('Dataframe loaded')


Loading dataframe


C:\Users\jorda\AppData\Local\Temp\ipykernel_10000\1872828216.py:30: DtypeWarning: Columns (33,35,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df: pd.DataFrame = pd.read_csv(file_path)


Dataframe loaded


## Privacy Budget

In [ ]:
rho: float = 1.0

In [77]:
'''
Differentially private mechanisms.
'''
def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def gaussian_mech_RDP_vec(vec, sensitivity, alpha, epsilon):
    sigma = np.sqrt((sensitivity**2 * alpha) / (2 * epsilon))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]

def gaussian_mech_zCDP_vec(vec, sensitivity, rho):
    sigma = np.sqrt((sensitivity**2) / (2 * rho))
    return [v + np.random.normal(loc=0, scale=sigma) for v in vec]

## Data Wrangling/Conversions

In [87]:
# As is, the unit of privacy is person-meet-division-age division

# 1. Remove all records which are not from a full-power division
# Drop any rows with NaN values in it.
df = df.dropna()

# 2. Limit to 1 record per meet (based on meet name/date for a person)
person_meet_columns: list[str] = ['Name', 'MeetName', 'Date']
df = df.drop_duplicates(subset=person_meet_columns, keep='first')

# 3. Convert person-meet unit of privacy to person-year
# Note: Person-state would protect a person while they reside in a specific state.
# Identifying in terms of how we determine a 'unique' person, all data can be identifying
# Full name + sex + competes in powerlifting is incredibly identifying so this is a proxy for a 'person'
df['Year'] = df['Date'].map(lambda x: int(x[:4]))
identifying_columns: list[str] = ['Name', 'Sex', 'Year']
histogram = df.groupby(identifying_columns).size()
noisy_histogram = gaussian_mech_zCDP_vec(histogram, 1, rho/2)

# This is our scalar value to divide rho by (max number of times a person has competed in a given year)
noisy_max: float = np.max(noisy_histogram)

8.293952244043492


In [88]:
# Drop Mx columns for a simplifying assumption
df = df[df['Sex'] != 'Mx']

# Fill empty values with 0 for untested
df['Tested'] = df['Tested'].fillna(0)

# Convert binary categorical columns into binary values
sex: dict = {'M': 1,'F': 0}
tested: dict = {'Yes': 1}
df['Tested'] = df['Tested'].map(tested)
df['Sex'] = df['Sex'].map(sex)

Convert each attempt into the attempt weight and another column for whether they made it or not.

In [89]:
# If an attempt was missed, it has a '-' put in front of it
# Separate each attempt into the weight loaded and whether it was successful.
attempt_columns: list[str] = ['Squat1Kg', 'Squat2Kg', 'Squat3Kg',
                              'Bench1Kg', 'Bench2Kg', 'Bench3Kg',
                              'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg']

for column in attempt_columns:
    df[f"{column}Made"] = df[column].map(lambda x: 1 if x > 0 else 0)
    df[column] = np.abs(df[column])

best_attempt_columns: list[str] = ['Best3SquatKg', 'Best3BenchKg', 'Best3DeadliftKg']

# If someone didn't hit any lifts, convert their best 3rd to 0
for column in best_attempt_columns:
    df[column] = df[column].map(lambda x: x if x > 0 else 0)


In [90]:
# Create one-hot encodings if they don't exist
categorical_columns: list[str] = ['Event', 'Equipment', 'ParentFederation']
if 'encoded_features' not in locals():
    # Create the One-Hot-Encoding
    encoded_features: list[pd.DataFrame] = [df[column].str.get_dummies("|") for column in categorical_columns if column in df.columns]

# Drop the categorical columns if they are in the dataframe
df = df.drop(categorical_columns, axis=1, errors='ignore')

# Concatenate one-hot-encoded columns along the column axis
for features in encoded_features:
    for column in features.columns:
        df[column] = features[column]

## Algorithms 

In [8]:
''' 
Machine learning functions (loss, gradient, noisy gradient descent).
'''

# The loss function measures how good our model is. The training goal is to minimize the loss.
# This is the logistic loss function.
def loss(theta, xi, yi):
    exponent = - yi * (xi.dot(theta))
    return np.log(1 + np.exp(exponent))

# This is the gradient of the logistic loss
# The gradient is a vector that indicates the rate of change of the loss in each direction
def gradient(theta, xi, yi):
    exponent = yi * (xi.dot(theta))
    return - (yi*xi) / (1+np.exp(exponent))

def avg_grad(theta, X, y):
    grads = [gradient(theta, xi, yi) for xi, yi in zip(X, y)]
    return np.mean(grads, axis=0)

# Prediction: take a model (theta) and a single example (xi) and return its predicted label
def predict(xi, theta, bias=0):
    label = np.sign(xi @ theta + bias)
    return label

def accuracy(theta, X_test, y_test):
    return np.sum(predict(X_test, theta) == y_test)/X_test.shape[0]

# L2 Clipping
def L2_clip(v, b):
    norm = np.linalg.norm(v, ord=2)
    
    if norm > b:
        return b * (v / norm)
    else:
        return v

def gradient_sum(theta, X, y, b):
    gradients = [L2_clip(gradient(theta, x_i, y_i), b) for x_i, y_i in zip(X,y)]
        
    # sum query
    # L2 sensitivity is b (by clipping performed above)
    return np.sum(gradients, axis=0)

'''
Noisy gradient descent algorithms.
'''
    
# Noisy gradient descent
# Satisfies (k*epsilon + epsilon, k*delta)-differential privacy
def noisy_gradient_descent(X_train, y_train, iterations, epsilon, delta):
    theta = np.zeros(X_train.shape[1])
    b = 3

    noisy_count = laplace_mech(X_train.shape[0], 1, epsilon)

    for i in range(iterations):
        clipped_gradient_sum = gradient_sum(theta, X_train, y_train, b)
        noisy_gradient_sum = np.array(gaussian_mech_vec(clipped_gradient_sum, b, epsilon, delta))
        noisy_avg_gradient = noisy_gradient_sum / noisy_count
        theta = theta - noisy_avg_gradient

    return theta

def noisy_gradient_descent_RDP(X_train, y_train, iterations, alpha, epsilon_bar):
    theta = np.zeros(X_train.shape[1])
    b = 3
    epsilon_bar_count = 0.05 * epsilon_bar
    epsilon_bar_i = 0.95 * epsilon_bar / iterations
    
    noisy_count = gaussian_mech_RDP_vec([len(X_train)], 1, alpha, epsilon_bar_count)[0]

    for i in range(iterations):
        clipped_gradient_sum = gradient_sum(theta, X_train, y_train, b)
        noisy_gradient_sum = np.array(gaussian_mech_RDP_vec(clipped_gradient_sum, b, alpha, epsilon_bar_i))
        noisy_avg_gradient = noisy_gradient_sum / noisy_count
        theta = theta - noisy_avg_gradient

    return theta

def noisy_gradient_descent_zCDP(X_train, y_train, iterations, rho):
    theta = np.zeros(X_train.shape[1])
    b = 3
    rho_count = 0.05 * rho
    rho_i = 0.95 * rho / iterations
  
    noisy_count = gaussian_mech_zCDP_vec([len(X_train)], 1, rho_count)[0]

    for i in range(iterations):
        clipped_gradient_sum = gradient_sum(theta, X_train, y_train, b)
        noisy_gradient_sum = np.array(gaussian_mech_zCDP_vec(clipped_gradient_sum, b, rho_i))
        noisy_avg_gradient = noisy_gradient_sum / noisy_count
        theta = theta - noisy_avg_gradient

    return theta

## Work

Predictive regression questions to answer:

* Sex
* Age
* Successful 3rd attempt
* Tested vs. untested
* Federation
* Equipment